# SCRAPING A LIST OF RESTAURANTS & PLACES TO VISIT IN AMSTERDAM FROM GOOGLE MAP LINK

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

In [2]:
chrome_options = webdriver.ChromeOptions()

In [3]:
chrome_options.binary_location = 'C:/chromedriver/chrome-win64/chrome.exe'
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument('--start-maximized')

In [4]:
service = Service('C:/chromedriver/chromedriver-win64/chromedriver.exe')

In [5]:
url = 'https://www.google.com/maps/d/u/0/viewer?mid=1IfRqpMMA8JBwH_XvIyxW8MYyzzg&ll=52.37821899999998%2C4.912948999999989&z=15'

In [6]:
restaurant_name = []
description = []
address = []
phone_number = []
website = []
ratings = []
is_restaurant = []

In [6]:
driver = webdriver.Chrome(service=service,options=chrome_options)

try:
    driver.get(url)    
    time.sleep(1)

    drop_down = driver.find_element(By.CLASS_NAME, 'HzV7m-pbTTYe-KoToPc-ornU0b')
    drop_down.click()

    time.sleep(1)
    top_level_div = driver.find_element(By.CLASS_NAME, 'HzV7m-pbTTYe-bN97Pc')

    # check if place is restaiurant or not
    places = top_level_div.find_elements(By.CLASS_NAME, 'pbTTYe-ibnC6b-Bz112c')
    for place in places:
        place = place.find_element(By.TAG_NAME, 'img')
        img_link = place.get_attribute('src')
        # print(img_link)
        if 'restaurant' in img_link:
            is_restaurant.append('yes')
        else:
            is_restaurant.append('no')

    # get all other details
    all_restaurants_div = top_level_div.find_elements(By.CSS_SELECTOR,'[class*="HzV7m-pbTTYe-ibnC6b pbTTYe-ibnC6b-d6wfac"]')

    name_and_desc_list = []
    for i, place in enumerate(all_restaurants_div):
        place.click()
        time.sleep(2)
        name_and_desc = driver.find_elements(By.CLASS_NAME, 'qqvbed-p83tee-lTBxed')

        # get nam and description
        # rname = details.find_element(By.XPATH, '//*[@id="featurecardPanel"]/div/div/div[4]/div[1]/div[1]/div[2]').text
        # rdesc = details.find_element(By.XPATH, '//*[@id="featurecardPanel"]/div/div/div[4]/div[1]/div[2]/div[2]').text
        if len(name_and_desc) == 2:
            for details in name_and_desc: 
                name_and_desc_list.append(details.text)                
            rname = name_and_desc_list[0]
            rdesc = name_and_desc_list[1]
            name_and_desc_list.clear()
        else:
            rname = driver.find_element(By.CLASS_NAME, 'qqvbed-p83tee-lTBxed').text
            rdesc = "None Provided"

        restaurant_name.append(rname)
        description.append(rdesc)
        
        # get address, phone number, ratings and website        
        try:  
            driver.implicitly_wait(1)
            top_details = driver.find_element(By.CSS_SELECTOR, 'div.qqvbed-VTkLkc.fO2voc-jRmmHf-LJTIlf')
            
            if top_details.is_displayed():
                try:
                    address_ = top_details.find_element(By.CLASS_NAME, 'fO2voc-jRmmHf-MZArnb-Q7Zjwb')
                    address_ = address_.text if address_.is_displayed() else 'None Provided'
                except Exception:
                    address_ = 'None Provided'
                
                try:
                    website_ = top_details.find_element(By.TAG_NAME, 'a')
                    website_ = website_.get_attribute('href') if website_.is_displayed() else 'None Provided'
                except Exception:
                    website_ = 'None Provided'
                
                try:
                    phone = top_details.find_element(By.XPATH, '//div[contains(text(), "+31 ")]')
                    phone = phone.text if phone.is_displayed() else 'None Provided'
                except Exception:
                    phone = 'None Provided'
                
                try:
                    rating = top_details.find_element(By.CLASS_NAME, 'fO2voc-jRmmHf-LJTIlf-wcwwM-H6j5tf')
                    rating = rating.text if rating.is_displayed() else 'None Provided'
                except Exception:
                    rating = 'None Provided'
            else:
                print("Top details section is not displayed.")
        except Exception as e:
            address_ = 'None Provided'
            website_ = 'None Provided'
            phone = 'None Provided'
            rating = 'None Provided'

        # append details
        address.append(address_)
        phone_number.append(phone)
        website.append(website_)
        ratings.append(rating)
        
        # Print extracted details
        print(f'{i}\t name: {rname}\n \t description: {rdesc}\n \t address: {address_}\n \t phone_number: {phone}\n \t website: {website_}\n \t rating: {rating}\n \t is_restaurant: {is_restaurant[i]} \n \t___________\n')
        
        # Navigate to the next item
        time.sleep(1)
        try:
            driver.find_element(By.CSS_SELECTOR, 'span.HzV7m-tJHJj-LgbsSe-Bz112c.qqvbed-a4fUwd-LgbsSe-Bz112c').click()
        except Exception as e:
            print(f"Navigation error: {e}")
except Exception as e:
    print(f'{e}')
finally:
    driver.quit()

## save scraped data to excel

In [10]:
print(len(restaurant_name))
print(len(description))
print(len(address))
print(len(phone_number))
print(len(website))
print(len(ratings))
print(len(is_restaurant))

90
90
90
90
90
90
90


In [11]:
data = {
    'name': restaurant_name,
    'description': description, 
    'address': address,
    'phone': phone_number,
    'website': website,
    'ratings': ratings,
    'is_restaurant': is_restaurant
}

In [12]:
scraped_to_df = pd.DataFrame(data)

In [13]:
scraped_to_df.to_excel('list of places to visit in amsterdam.xlsx', index=False, engine='openpyxl')

## read data from excel

In [14]:
read_data = pd.read_excel('list of places to visit in amsterdam.xlsx')

In [15]:
read_data.head()

,name,description,address,phone,website,ratings,is_restaurant
0,Greenhouse,smokey shop,"Tolstraat 91, 1074 VG Amsterdam, Netherlands",None Provided,https://greenhousecoffeeshops.com/,4.3,no
1,Amstel 288,"1017 Amsterdam, The Netherlands",None Provided,None Provided,None Provided,None Provided,no
2,Barney's Breakfast Bar,None Provided,None Provided,None Provided,None Provided,None Provided,no
3,Abraxas,"Sint AntoniesbreestraatAmsterdam, The Netherlands",None Provided,None Provided,None Provided,None Provided,no
4,"Horecaexploitatiemaatschappij ""De Klos"" B.V.",Best ribs in Europe; big meal; recomm by Michael;,"Kerkstraat 41, 1017 GB Amsterdam, Netherlands",+31 20 625 3730,https://dekloscafe.wordpress.com/,4.5,yes


In [17]:
read_data.tail()

,name,description,address,phone,website,ratings,is_restaurant
85,Restaurant d'Vijff Vlieghen,perhaps the most famous spot in town for tradi...,"Spuistraat 294-302, 1012 VX Amsterdam, Netherl...",+31 20 530 4060,http://vijffvlieghen.nl/,4.3,yes
86,"Wijn- en Spijslokaal ""De Witte Uyl""",Amsterdam local fav restaurant,"Frans Halsstraat 26, 1072 BR Amsterdam, Nether...",+31 20 670 0458,None Provided,4.0,yes
87,Restaurant Moeders,good Dutch cooking,"Rozengracht 251, 1016 SX Amsterdam, Netherlands",+31 20 626 7957,https://mymenuweb.com/fr/restaurants/831051/?u...,4.5,yes
88,Hotel Hoksbergen,None Provided,"Singel 301, 1012 WH Amsterdam, Netherlands",+31 20 626 6043,http://hotelhoksbergen.com/,3.6,no
89,Hotel Hoksbergen,None Provided,"Singel 301, 1012 WH Amsterdam, Netherlands",+31 20 626 6043,http://hotelhoksbergen.com/,3.6,no
